In [1]:
!pip install pennylane

     |████████████████████████████████| 409kB 6.0MB/s 


In [1]:
import time
import os
import copy


import pennylane as qml
from pennylane import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, transforms


import matplotlib.pyplot as plt

In [2]:
class_names = ['airplane','automobile','bird','cat','deer',
               'dog','frog','horse','ship','truck']
#These are the classes in the dataset

In [3]:
data_path = r"/content/drive/My Drive/cifar10/data"

cifar10 = datasets.CIFAR10(
    data_path, train=True, download=True,
    transform=transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225]) # mean and std from resnet18
    ]))


cifar10_val = datasets.CIFAR10(
    data_path, train=False, download=True,
    transform=transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
    ]))

Extracting /content/drive/My Drive/cifar10/data/cifar-10-python.tar.gz to /content/drive/My Drive/cifar10/data
Files already downloaded and verified


In [4]:
import random
class_names = ['airplane', 'bird']
label_map = {0: 0, 2: 1}

cifar2_train = [(img, label_map[label])
          for img, label in cifar10
          if label in [0, 2]]

cifar2_val = [(img, label_map[label])
              for img, label in cifar10_val
              if label in [0, 2]]
              
nt = len(cifar2_train) 
nv = len(cifar2_val)
n_train = int(400)  #Edit: test and val data batches are the same size, for more consistent validation
n_val = int(400)
idxT = list(range(nt)) 
idxV = list(range(nv))  
random.shuffle(idxT)  
random.shuffle(idxV)

train_idx = idxT[:n_train]
val_idx = idxV[:n_val]

In [5]:
from torch.utils import data
image_datasets_raw = {'train': cifar2_train, 'val': cifar2_val}
image_datasets = {x: data.Subset(image_datasets_raw[x], train_idx if x == 'train' else val_idx) for x in image_datasets_raw }


dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4, shuffle=True, num_workers=2) for x in ['train', 'val']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

device = (torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu"))
print(f"Training on device {device}.")
use_cuda = torch.cuda.is_available()

Training on device cpu.


In [6]:
def train_model(model, loss_fn, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train() 
            else:
                model.eval()  

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

               #forward pass
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1) # inp, dim
                    loss = loss_fn(outputs, labels)

                    # backwards pass
                    if phase == 'train':
                        loss.backward() # calculate gradiens
                        optimizer.step() # update params

                #stats
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print('\n')

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load weights of best model
    model.load_state_dict(best_model_wts)
    return model

In [7]:
#Some hyperparameters 

n_qubits = 4                
step = 0.0004               
batch_size = 4                        
q_depth = 6                 
q_delta = 0.01                              

In [8]:
#if this throws exception, restart runtime, but do not reinstall pennylane package!!!

dev = qml.device("default.qubit", n_qubits) # name of device, number of qubits
#Any computational object that can apply quantum operations, and return an measurement value is called a quantum device.
# in this example i use the default.qubit device, which is a pure qubit state device

In [9]:
# quantumn functions
def H_layer(nqubits): # superposition func
    for idx in range(nqubits): # loop for each qubit in system
        qml.Hadamard(wires=idx) # apply the h-gate on this qubit


def RY_layer(w):# parametirized qubit rotations around the y axis
    for idx, element in enumerate(w):
        qml.RY(element, wires=idx)

#layer for entanglement
def entangling_layer(nqubits):
    for i in range(0, nqubits - 1, 2):  
        qml.CNOT(wires=[i, i + 1])
    for i in range(1, nqubits - 1, 2):  
        qml.CNOT(wires=[i, i + 1])

In [10]:
#this is merely for showing how the entangeling is taking place. It has no apllication other than that.

qbits= 4
for i in range(0, qbits - 1, 2):  # Loop over even indices: i=0,2,...N-2
        print(f'qubit {i} and tarbit {i+1}')
for i in range(1, qbits - 1, 2):  # Loop over odd indices:  i=1,3,...N-3
        print(f'cnbit {i} and tarbit {i+1}')

qubit 0 and tarbit 1
qubit 2 and tarbit 3
cnbit 1 and tarbit 2


q0-q1-q2-q3

In [11]:
#quantum node 
@qml.qnode(dev, interface="torch")  # quantum device, pytorch interface
def quantum_net(q_input_features, q_weights_flat): # out_tensor,  weight

    # Reshape weights
    q_weights = q_weights_flat.reshape(q_depth, n_qubits) # Returns a tensor with the same data and number of elements as input, but with the specified shape(6 rows 4 columns)

    H_layer(n_qubits) #superposition

    # Embed features in the quantum node
    RY_layer(q_input_features) # rotation around y-axis based on q_input_features

    # Sequence of trainable variational layers
    for k in range(q_depth):
        entangling_layer(n_qubits)
        RY_layer(q_weights[k])

    # Expectation values in the Z basis
    exp_vals = [qml.expval(qml.PauliZ(position)) for position in range(n_qubits)] # measuring with the Z operator produces a classical output vector, suitable for additional post-processing.
    return tuple(exp_vals) 

In [12]:
class QuantumNet(nn.Module): # subclass of the nn.Module which is the pytorch network module
    """
    Torch module implementing the quantum network
    """

    def __init__(self):
        """
        This is the structure of the network 
        """

        super().__init__()
        self.pre_net = nn.Linear(512, n_qubits) # input 512, out 4
        self.q_params = nn.Parameter(q_delta * torch.randn(q_depth * n_qubits)) # 0.01 * torch.rand(6*4) # add to module parameters
        self.post_net = nn.Linear(n_qubits, 2) # last layer

    def forward(self, input_features):
        """
        This is how the tensors are moving thought the quantum network
        """

        # obtain the input features for the quantum circuit
        # by reducing the feature dimension from 512 to 4
        pre_out = self.pre_net(input_features) # look in init
        q_in = torch.tanh(pre_out) * np.pi / 2.0 #activation function. with constant scaling by np.pi/2.0

        # Apply the quantum circuit to each element of the batch and append to q_out
        q_out = torch.Tensor(0, n_qubits) # tensor of size [0,4]
        q_out = q_out.to(device) # sends q_out to device (cpu or gpu)
        for elem in q_in: # 4 elements
            q_out_elem = quantum_net(elem, self.q_params).float().unsqueeze(0) # 
            q_out = torch.cat((q_out, q_out_elem)) # concatinates and therby fills the empty tensor q_out in

        # return the two-dimensional prediction from the postprocessing layer
        return self.post_net(q_out) #returns the output of the last layer

In [13]:
model_hybrid = torchvision.models.resnet18(pretrained=True)

for param in model_hybrid.parameters(): # sets paramters of model to not be updated by grad
    param.requires_grad = False 


# sets last layer of the model to be the quantum layer
model_hybrid.fc = QuantumNet()

# send the model to cuda or cpu according to the "device" object.
#pennylane currently does not support cuda, so we have to use a cpu, as of now.
model_hybrid = model_hybrid.to(device)

loss_fn = nn.CrossEntropyLoss() # loss function that combines negative likelihood loss and softmax

optimizer_hybrid = optim.Adam(model_hybrid.fc.parameters(), lr=step) #step = 0.0004  

exp_lr_scheduler = lr_scheduler.StepLR( # each 7 steps decay lr by a factor of 0.1
    optimizer_hybrid, step_size=7, gamma= 0.1
)


Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth


In [ ]:
model_hybrid = train_model(
    model = model_hybrid, loss_fn = loss_fn, optimizer = optimizer_hybrid, scheduler = exp_lr_scheduler, num_epochs=30
)

In [ ]:
#accuracy after 30 epochs 95% on vapl data.

In [ ]:
"""The reason - if you wonder- that the val loss is significantly less than the train loss,
 is because of the data augmentation that are used during training, but are not used during validation."""